In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
import plotly.express as px
import numpy as np

Dữ liệu ban đầu được chia thành các sheet khác nhau để dễ xử lý: "revenue", "cogs" và "fee"

In [2]:
def readData(sheetName="", valueName=""):
    data = pd.read_excel("reti_data.xlsx", sheet_name=sheetName, header=None).transpose()
    data.columns = data.iloc[0]
    data = data[1:]
    df = pd.melt(
        data,
        id_vars=['Tháng',"Năm"],
        value_vars=data.columns[2:],
        var_name="project",
        value_name=valueName
    ).rename(columns={
        "Tháng":"month",
        "Năm":"year"
    }).assign(
        date=lambda x: x['month'].astype(int).astype(str) + '/' + x['year'].astype(int).astype(str)
    ).assign(
        date = lambda x: pd.to_datetime(x['date'], format="%m/%Y") + MonthEnd(0)
    )
    return df

In [ ]:
rev_df = readData('revenue', 'revenue')
cogs_df = readData('cogs', 'cogs')
rev_df["project"] = rev_df["project"].replace("DOANH THU", "total")
cogs_df["project"] = cogs_df["project"].replace("GIÁ VỐN HÀNG BÁN", "total")

In [14]:
pd.merge(rev_df, cogs_df, how='outer', on=["project", "date", "year", "month"]).assign(
    CoR = lambda x: x["cogs"] / x["revenue"].replace(0, np.nan)
)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11124\1053591871.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  CoR = lambda x: x["cogs"] / x["revenue"].replace(0, np.nan)


,month,year,project,revenue,date,cogs,CoR
0,1.0,2024.0,Doanh thu cho thuê HCM,89091345.0,2024-01-31,NaN,NaN
1,2.0,2024.0,Doanh thu cho thuê HCM,88749053.0,2024-02-29,NaN,NaN
2,3.0,2024.0,Doanh thu cho thuê HCM,5292206.0,2024-03-31,NaN,NaN
3,4.0,2024.0,Doanh thu cho thuê HCM,0.0,2024-04-30,NaN,NaN
4,5.0,2024.0,Doanh thu cho thuê HCM,0.0,2024-05-31,NaN,NaN
...,...,...,...,...,...,...,...
485,10.0,2024.0,total,16667480507.249197,2024-10-31,10742338881.197002,0.644509
486,11.0,2024.0,total,4107057676.366505,2024-11-30,2798482521.2985,0.681384
487,12.0,2024.0,total,10902849307.128,2024-12-31,5967661498.887801,0.547349
488,1.0,2025.0,total,683449920.203,2025-01-31,427551308.5421,0.625578


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10760\63151610.py:2: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,month,year,project,revenue,date,cogs,CoR
0,1.0,2024.0,DOANH THU,333406148.0,2024-01-31,NaN,NaN
1,1.0,2025.0,DOANH THU,683449920.203,2025-01-31,NaN,NaN
2,2.0,2024.0,DOANH THU,88749053.0,2024-02-29,NaN,NaN
3,2.0,2025.0,DOANH THU,1382445966.785,2025-02-28,NaN,NaN
4,3.0,2024.0,DOANH THU,325324554.154,2024-03-31,NaN,NaN
...,...,...,...,...,...,...,...
471,8.0,2024.0,Vinhomes Royal Residences,0.0,2024-08-31,0.0,NaN
472,9.0,2024.0,Vinhomes Royal Residences,0.0,2024-09-30,0.0,NaN
473,10.0,2024.0,Vinhomes Royal Residences,146894178.34,2024-10-31,104924413.1,0.714286
474,11.0,2024.0,Vinhomes Royal Residences,60240202.26,2024-11-30,43028715.9,0.714286


In [ ]:
1293696913.785

In [94]:
df.sort_values(by=["year", "month"]).loc[df['project']=="DOANH THU"]["revenue"].diff(periods=12).tail(1).values[0]

1293696913.785

In [33]:
np.nansum(df.loc[~df.project.isin(["DOANH THU"])].loc[df['year']==2024]["revenue"])

78228104524.1061

In [19]:
px.bar(
    data_frame=df.loc[~df.project.isin(["DOANH THU", "Thưởng nóng", "Doanh thu cho thuê HCM"])], 
    x="date", y="revenue",
    color="project",
    barmode="stack"
)